In [ ]:
#Step 1: Setting Up the Environment
#Install Required Packages: Make sure you have Python installed, then create a virtual environment and install the required packages.
python -m venv venv
source venv/bin/activate  # On Windows use `venv\Scripts\activate`
pip install Flask Flask-JWT-Extended Flask-SQLAlchemy Flask-Migrate Flask-Marshmallow marshmallow-sqlalchemy
#Project Structure: Create a folder structure as follows:
employee_management/
├── app.py
├── models.py
├── schemas.py
├── config.py
├── requirements.txt
├── migrations/
└── venv/

In [ ]:
#Step 2: Configuration
#Create a config.py file to hold the configuration for the Flask app.
# config.py
import os

class Config:
    SQLALCHEMY_DATABASE_URI = 'sqlite:///employees.db'
    SQLALCHEMY_TRACK_MODIFICATIONS = False
    JWT_SECRET_KEY = 'your_jwt_secret_key'  # Change this to a random secret key

In [ ]:
#Step 3: Models
#Create a models.py file to define the Employee model.
# models.py
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime

db = SQLAlchemy()

class Employee(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    department = db.Column(db.String(50))
    role = db.Column(db.String(50))
    date_joined = db.Column(db.DateTime, default=datetime.utcnow)

    def __repr__(self):
        return f'<Employee {self.name}>'

In [ ]:
#Step 4: Schemas
#Create a schemas.py file for serialization.
# schemas.py
from marshmallow_sqlalchemy import SQLAlchemyAutoSchema
from models import Employee

class EmployeeSchema(SQLAlchemyAutoSchema):
    class Meta:
        model = Employee
        load_instance = True

In [ ]:
#Step 5: Main Application Logic
#Create an app.py file for the main application logic.
# app.py
from flask import Flask, request, jsonify
from flask_jwt_extended import JWTManager, create_access_token, jwt_required
from models import db, Employee
from schemas import EmployeeSchema
from config import Config

app = Flask(__name__)
app.config.from_object(Config)
db.init_app(app)
jwt = JWTManager(app)

# Create the database
with app.app_context():
    db.create_all()

# Authentication endpoint
@app.route('/api/auth', methods=['POST'])
def authenticate():
    username = request.json.get('username')
    password = request.json.get('password')
    # In a real application, validate username and password
    access_token = create_access_token(identity=username)
    return jsonify(access_token=access_token)

# Create an Employee
@app.route('/api/employees/', methods=['POST'])
@jwt_required()
def create_employee():
    schema = EmployeeSchema()
    employee = schema.load(request.json)
    db.session.add(employee)
    db.session.commit()
    return schema.dump(employee), 201

# List all Employees
@app.route('/api/employees/', methods=['GET'])
@jwt_required()
def list_employees():
    page = request.args.get('page', 1, type=int)
    department = request.args.get('department', None)
    role = request.args.get('role', None)

    query = Employee.query
    if department:
        query = query.filter_by(department=department)
    if role:
        query = query.filter_by(role=role)

    employees = query.paginate(page=page, per_page=10)
    schema = EmployeeSchema(many=True)
    return jsonify(schema.dump(employees.items)), 200

# Retrieve a Single Employee
@app.route('/api/employees/<int:id>/', methods=['GET'])
@jwt_required()
def get_employee(id):
    employee = Employee.query.get_or_404(id)
    schema = EmployeeSchema()
    return schema.dump(employee), 200

# Update an Employee
@app.route('/api/employees/<int:id>/', methods=['PUT'])
@jwt_required()
def update_employee(id):
    employee = Employee.query.get_or_404(id)
    schema = EmployeeSchema()
    updated_employee = schema.load(request.json, instance=employee, partial=True)  # Allow partial updates
    db.session.commit()
    return schema.dump(updated_employee), 200

# Delete an Employee
@app.route('/api/employees/<int:id>/', methods=['DELETE'])
@jwt_required()
def delete_employee(id):
    employee = Employee.query.get_or_404(id)
    db.session.delete(employee)
    db.session.commit()
    return '', 204  # No content

In [ ]:
#Step 6: Running the Application
#To run the application, add the following block at the end of the app.py file:
if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
#Step 7: Testing the API
#You can use Postman to test the API endpoints. Here’s how to do it:

#1: Start the Flask Server: Run the application in your terminal.
python app.py

#Authenticate:

#2: Send a POST request to /api/auth with a JSON body containing the username and password.
#Example JSON body:
{
  "username": "your_username",
  "password": "your_password"
}
#You will receive a token in the response.

#3: Add the Token to Postman:

#Go to the "Authorization" tab in Postman for each request that requires authentication.
#Select "Bearer Token" and paste the token you received from the authentication step.

#4: Create Employee:

#Send a POST request to /api/employees/ with a JSON body containing employee details.
#Example JSON body:
{
  "name": "John Doe",
  "email": "john.doe@example.com",
  "department": "Engineering",
  "role": "Developer"
}

#5: List Employees:

#Send a GET request to /api/employees/ to retrieve all employees.
#You can add query parameters for filtering (e.g., ?department=Engineering) and pagination (e.g., ?page=1).

#6: Retrieve Single Employee:

#Send a GET request to /api/employees/{id}/ to retrieve an employee by ID.

#7: Update Employee:

#Send a PUT request to /api/employees/{id}/ with the updated employee details in the JSON body.

#8: Delete Employee:

#Send a DELETE request to /api/employees/{id}/ to delete an employee.


In [ ]:
#Step 8: Testing with Unit Tests
## test_app.py
import unittest
import json
from app import app, db

class EmployeeAPITestCase(unittest.TestCase):
    def setUp(self):
        self.app = app
        self.client = self.app.test_client()
        self.app.config['TESTING'] = True
        self.app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///:memory:'
        with self.app.app_context():
            db.create_all()

    def tearDown(self):
        with self.app.app_context():
            db.session.remove()
            db.drop_all()

    def test_create_employee(self):
        response = self.client.post('/api/employees/', json={
            "name": "Jane Doe",
            "email": "jane.doe@example.com",
            "department": "HR",
            "role": "Manager"
        })
        self.assertEqual(response.status_code, 201)

    def test_list_employees(self):
        response = self.client.get('/api/employees/')
        self.assertEqual(response.status_code, 200)

    def test_get_employee(self):
        response = self.client.post('/api/employees/', json={
            "name": "John Smith",
            "email": "john.smith@example.com",
            "department": "Engineering",
            "role": "Developer"
        })
        employee_id = json.loads(response.data)['id']
        response = self.client.get(f'/api/employees/{employee_id}/')
        self.assertEqual(response.status_code, 200)

        def test_update_employee(self):
        # Create an employee to update
        response = self.client.post('/api/employees/', json={
            "name": "John Smith",
            "email": "john.smith@example.com",
            "department": "Engineering",
            "role": "Developer"
        })
        employee_id = json.loads(response.data)['id']

        # Update the employee's information
        response = self.client.put(f'/api/employees/{employee_id}/', json={
            "role": "Senior Developer"
        })
        self.assertEqual(response.status_code, 200)
        self.assertIn("Senior Developer", str(response.data))

    def test_delete_employee(self):
        # Create an employee to delete
        response = self.client.post('/api/employees/', json={
            "name": "Alice Johnson",
            "email": "alice.johnson@example.com",
            "department": "Sales",
            "role": "Sales Representative"
        })
        employee_id = json.loads(response.data)['id']

        # Delete the employee
        response = self.client.delete(f'/api/employees/{employee_id}/')
        self.assertEqual(response.status_code, 204)

        # Try to get the deleted employee
        response = self.client.get(f'/api/employees/{employee_id}/')
        self.assertEqual(response.status_code, 404)

if __name__ == '__main__':
    unittest.main()


In [ ]:
#Step 9: Running the Tests
#Run the Tests: You can run the tests using the following command in your terminal:
python -m unittest test_app.py
#This will execute all the test cases defined in test_app.py. You should see output indicating whether the tests passed or failed.